In [19]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin
import json, requests, pickle

import smtplib
import os
from email.mime.text import MIMEText
from email import encoders

In [20]:
# Google 아이디, 비밀번호 저장 파일
EMAIL_USER_DATA_SAVED_FILE = 'res/account'

In [15]:
# 계정 정보를 저장하기 위해 최초로 한 번만 동작시키도록 한다.
# 이후로는 또 동작하면(계정 정보가 다시 리셋)
# 밑에 uerID와 userpw부분은 실제 계정 이름과 비번으로 등록 해주세요. 한번만 하면 되니 그다음부터 이 단락은 안돌려도 됩니다. 여기는 account에 정보
# 넣는 창입니다.
import pickle

email_user_info = dict({
    "smtp_user_id": "wsk0307@naver.com",
    "smtp_user_pw": ""
})

f = open(EMAIL_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(email_user_info, f)
f.close()

In [21]:
f = open(EMAIL_USER_DATA_SAVED_FILE, 'rb')
email_user_info = pickle.load(f)
f.close()

In [22]:
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info['smtp_server'], smtp_info['smtp_port']) as server:
        server.starttls()
        
        server.login(email_user_info['smtp_user_id'], email_user_info['smtp_user_pw'])
        
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공')
        
            return 'success'
        else:
            print(res)
 
            return 'fail'

In [23]:
import datetime

app = Flask(__name__)
CORS(app)

@app.route('/findPwByEmail', methods = ['GET', 'POST'])
def findPwByEmail():
    params = json.loads(request.get_data(), encoding='utf-8')
    
    name = params['name']
    email = params['email']
    
    print(name, email)
#네이버든 구글이든 서버 포트는 동일하게 587입니다. 그리고 그전에 네이버 SMTP 사용함으로 바꿔주시고 gmail도 보안 낮은 앱 사용으로 바꿔주세요
#구글 보안 링크 https://myaccount.google.com/lesssecureapps?pli=1&rapt=AEjHL4OKOBb13PWoNRnv2K9j0JNt7z9hJP2c2ZMV8Bnzt7pB1E9pqoh554QydK6ldj5wKnTqWSfYahmtn7bHRGuc_LeeuRbJhA
    smtp_info = dict({
        'smtp_server': 'smtp.naver.com', 
        'smtp_port': 587
    })

    now = datetime.datetime.now()
    now_time = now.strftime('%Y-%m-%d %H:%M')
    
    title = '유러너 - ' + name + '님 비밀번호 재설정 링크입니다.'
    content =  now_time + '에 비밀번호 재설정을 요청하였습니다.\n아래 링크에서 비밀번호 재설정을 완료해 주세요.\http://localhost:8080/changepw?name=' + name 
    sender = email_user_info['smtp_user_id']
    receiver = email
    
    msg = MIMEText(_text = content, _charset = 'utf-8')

    msg['Subject'] = title
    msg['From'] = sender
    msg['To'] = receiver
    
    isSuccess = send_email(smtp_info, msg)

    print(isSuccess)
    
    return isSuccess

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
<ipython-input-23-ca77754b5732>:8: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  params = json.loads(request.get_data(), encoding='utf-8')


wonkim wsk0307@naver.com


127.0.0.1 - - [03/Nov/2021 20:08:30] "POST /findPwByEmail HTTP/1.1" 200 -


이메일 전송 성공
success
